# 🦞 CoLobster

Run your own **OpenClaw** AI gateway on a free Google Colab instance — 3 clicks and you're live.

---
## Step 1 — Add Your API Key(s)

OpenClaw needs at least **one** LLM provider key. You set them up once in Colab’s secure Secrets storage — they persist across sessions and never appear in the notebook.

### First time? Here’s how:

1. Open the **🔑 Secrets** panel — click the key icon in the left sidebar
2. Click **"+ Add new secret"**
3. Type the **Name** exactly as shown below, paste your key as the **Value**
4. Toggle **"Notebook access"** ON

Add any of these (you only need one):

| Name (copy exactly) | Provider | Get a key | Free tier? |
|---|---|---|---|
| `GEMINI_API_KEY` | Google Gemini | [aistudio.google.com/apikey](https://aistudio.google.com/apikey) | ✅ Yes |
| `OPENROUTER_API_KEY` | OpenRouter | [openrouter.ai/keys](https://openrouter.ai/keys) | ✅ Some models |
| `ANTHROPIC_API_KEY` | Anthropic (Claude) | [console.anthropic.com](https://console.anthropic.com/settings/keys) | ❌ Pay-as-you-go |
| `OPENAI_API_KEY` | OpenAI (GPT) | [platform.openai.com/api-keys](https://platform.openai.com/api-keys) | ❌ Pay-as-you-go |

> 💡 **Easiest free start:** Get a Gemini key — it’s free, takes 30 seconds, and works great.

### Already done this before?

Your secrets are saved in Colab. Just make sure **Notebook access** is toggled ON for this notebook, then move on to Step 2.

---
## Step 2 — Install & Configure

This cell installs Node.js + OpenClaw and detects your API keys. Takes a few minutes on first run.

In [ ]:
#@title ▶️ Click to install & configure
import subprocess, os, secrets, sys, time, json
from IPython.display import display, HTML

# ============================================================
# Phase 1: Install Node.js 22 + OpenClaw
# ============================================================
display(HTML('<b style="color:#333">⏳ Installing Node.js & OpenClaw...</b>'))

# Check Node.js
node_ok = False
try:
    ver = subprocess.run(['node', '-v'], capture_output=True, text=True)
    major = int(ver.stdout.strip().lstrip('v').split('.')[0])
    node_ok = major >= 22
except Exception:
    pass

if not node_ok:
    subprocess.run('curl -fsSL https://deb.nodesource.com/setup_22.x | sudo -E bash -',
                   shell=True, capture_output=True)
    subprocess.run(['sudo', 'apt-get', 'install', '-y', 'nodejs'], capture_output=True)

# Check OpenClaw
oc = subprocess.run(['which', 'openclaw'], capture_output=True, text=True)
if not oc.stdout.strip():
    r = subprocess.run(['npm', 'install', '-g', 'openclaw@latest'], capture_output=True, text=True)
    if r.returncode != 0:
        print(r.stderr[-1000:])
        raise RuntimeError('npm install failed')

node_v = subprocess.run(['node', '-v'], capture_output=True, text=True).stdout.strip()
oc_bin = subprocess.run(['which', 'openclaw'], capture_output=True, text=True).stdout.strip()

display(HTML(f'''
<div style="background:#e8f5e9; color:#1b5e20; border-radius:6px; padding:8px 14px; margin:4px 0; font-size:13px;">
  ✅ Node.js {node_v} &nbsp;·&nbsp; OpenClaw at <code style="color:#2e7d32">{oc_bin}</code>
</div>
'''))

# ============================================================
# Phase 2: Detect API keys from Colab Secrets
# ============================================================
try:
    from google.colab import userdata
    _has_colab = True
except ImportError:
    _has_colab = False

KEY_NAMES = ['ANTHROPIC_API_KEY', 'OPENAI_API_KEY', 'GEMINI_API_KEY', 'OPENROUTER_API_KEY']

found = {}
for name in KEY_NAMES:
    val = None
    if _has_colab:
        try:
            val = userdata.get(name)
        except Exception:
            pass
    if not val:
        val = os.environ.get(name)
    if val and val.strip():
        found[name] = val.strip()
        os.environ[name] = val.strip()

if not found:
    display(HTML('''
    <div style="background:#fff3cd; color:#856404; border:1px solid #ffc107; border-radius:8px; padding:16px; margin:12px 0;">
      <b>⚠️ No API keys found.</b><br><br>
      Go back to <b>Step 1</b> and add at least one key in the 🔑 Secrets panel.<br>
      Make sure <b>"Notebook access"</b> is toggled ON, then re-run this cell.
    </div>
    '''))
else:
    # Show what we found
    provider_names = {
        'ANTHROPIC_API_KEY': 'Anthropic',
        'OPENAI_API_KEY': 'OpenAI',
        'GEMINI_API_KEY': 'Gemini',
        'OPENROUTER_API_KEY': 'OpenRouter',
    }
    detected = [provider_names[k] for k in found]

    # ============================================================
    # Phase 3: Prepare state dir, .env & config
    # ============================================================
    STATE_DIR = os.environ.get('OPENCLAW_STATE_DIR', '/content/openclaw_state')
    os.makedirs(STATE_DIR, exist_ok=True)
    os.environ['OPENCLAW_STATE_DIR'] = STATE_DIR

    token = secrets.token_hex(32)
    os.environ['OPENCLAW_GATEWAY_TOKEN'] = token

    # Write .env with API keys
    with open(os.path.join(STATE_DIR, '.env'), 'w') as f:
        f.write(f'OPENCLAW_GATEWAY_TOKEN={token}\n')
        for k, v in found.items():
            f.write(f'{k}={v}\n')

    display(HTML(f'''
    <div style="background:#d4edda; color:#155724; border:1px solid #28a745; border-radius:8px; padding:12px 16px; margin:8px 0;">
      ✅ <b>Ready!</b> Keys detected: {', '.join(detected)}<br>
      <span style="font-size:13px;">OpenClaw will automatically pick the best model for your keys. Run <b>Step 3</b> to start.</span>
    </div>
    '''))


---
### Choose a Model (optional)

OpenClaw auto-selects a model from your available keys, but defaults to **Claude Opus 4.6** — which is powerful but expensive.

Run the cell below to pick a different model. If you skip this, OpenClaw uses its default.

> 💡 **Cheapest start:** Gemini 3 Flash is free and fast. OpenRouter also gives access to models from DeepSeek, Moonshot (Kimi), xAI (Grok) and more.

In [ ]:
#@title ▶️ Choose a model
import os
import ipywidgets as widgets
from IPython.display import display, HTML

# Available models per provider key (curated list — updated Feb 2026)
PROVIDER_MODELS = {
    'GEMINI_API_KEY': [
        ('Gemini 3 Flash Preview  —  free, fast', 'google/gemini-3-flash-preview'),
        ('Gemini 3.1 Pro Preview  —  free, powerful', 'google/gemini-3.1-pro-preview'),
    ],
    'ANTHROPIC_API_KEY': [
        ('Claude Sonnet 4.6  —  balanced', 'anthropic/claude-sonnet-4-6'),
        ('Claude Haiku 4.5  —  fast, cheap', 'anthropic/claude-haiku-4-5'),
        ('Claude Opus 4.6  —  most capable, expensive', 'anthropic/claude-opus-4-6'),
    ],
    'OPENAI_API_KEY': [
        ('GPT-5 Nano  —  fast, cheap', 'openai/gpt-5-nano'),
        ('GPT-5.2  —  balanced', 'openai/gpt-5.2'),
        ('GPT-5.2 Pro  —  most capable, expensive', 'openai/gpt-5.2-pro'),
    ],
    'OPENROUTER_API_KEY': [
        ('Gemini 3 Flash (via OR)  —  free', 'openrouter/google/gemini-3-flash-preview'),
        ('DeepSeek V3.2 (via OR)  —  fast, cheap', 'openrouter/deepseek/deepseek-v3.2'),
        ('Kimi K2.5 (via OR)  —  powerful', 'openrouter/moonshotai/kimi-k2.5'),
        ('Grok 4.1 Fast (via OR)', 'openrouter/x-ai/grok-4.1-fast'),
        ('Qwen 3.5 Plus (via OR)', 'openrouter/qwen/qwen3.5-plus-02-15'),
        ('MiniMax M2.5 (via OR)', 'openrouter/minimax/minimax-m2.5'),
    ],
}

# Build options from detected keys
options = []
for key_name in ['GEMINI_API_KEY', 'ANTHROPIC_API_KEY', 'OPENAI_API_KEY', 'OPENROUTER_API_KEY']:
    if os.environ.get(key_name):
        options.extend(PROVIDER_MODELS[key_name])

if not options:
    display(HTML('''
    <div style="background:#fff3cd; color:#856404; border:1px solid #ffc107; border-radius:8px; padding:14px;">
      <b>⚠️</b> No API keys detected. Run <b>Step 2</b> first, then come back here.
    </div>
    '''))
else:
    # Pre-select the first option (cheapest/free model for the first detected provider)
    dropdown = widgets.Dropdown(
        options=options,
        value=options[0][1],
        description='',
        layout=widgets.Layout(width='450px'),
        style={'description_width': '0px'},
    )

    def on_change(change):
        os.environ['OPENCLAW_SELECTED_MODEL'] = change['new']

    dropdown.observe(on_change, names='value')

    # Set initial selection
    os.environ['OPENCLAW_SELECTED_MODEL'] = dropdown.value

    display(HTML('<b style="font-size:14px; color:#333;">Select a model:</b>'))
    display(dropdown)
    display(HTML(f'''
    <div style="background:#e8f5e9; color:#1b5e20; border-radius:6px; padding:8px 14px; margin:8px 0; font-size:13px;">
      ✅ Selected: <code style="color:#2e7d32">{dropdown.value}</code> — this will be used when you run Step 3.
      <br><span style="color:#555;">Change the dropdown above to pick a different model. You can also change it later via the dashboard.</span>
    </div>
    '''))

---
### Optional: Google Drive Persistence

By default, OpenClaw state (sessions, config) is stored locally and **lost when this Colab runtime shuts down** (after ~12 hours of inactivity, or when you disconnect).

If you want your conversations and settings to survive between sessions, run this cell **before** starting the gateway. It mounts Google Drive and stores OpenClaw state there.

> Skip this if you're just trying things out — you can always come back and enable it later (you'll need to re-run Steps 2 and 3).

In [ ]:
#@title ▶️ Enable Drive storage (optional)
from google.colab import drive
import os, shutil
from IPython.display import display, HTML

drive.mount('/content/drive')

DRIVE = '/content/drive/MyDrive/openclaw/state'
LOCAL = '/content/openclaw_state'
os.makedirs(DRIVE, exist_ok=True)

# Copy any existing local state to Drive
if os.path.exists(LOCAL):
    for item in os.listdir(LOCAL):
        s, d = os.path.join(LOCAL, item), os.path.join(DRIVE, item)
        if os.path.isfile(s): shutil.copy2(s, d)
        elif os.path.isdir(s) and not os.path.exists(d): shutil.copytree(s, d)

os.environ['OPENCLAW_STATE_DIR'] = DRIVE
link = '/content/.openclaw'
if os.path.islink(link): os.unlink(link)
os.symlink(DRIVE, link)

display(HTML('''
<div style="background:#d4edda; color:#155724; border:1px solid #28a745; border-radius:8px; padding:14px;">
  ✅ <b>Drive persistence enabled.</b> State will be saved to Google Drive.<br>
  <span style="font-size:13px;">Now run <b>Step 3</b> to start OpenClaw.</span>
</div>
'''))


---
## Step 3 — Start OpenClaw

In [ ]:
#@title ▶️ Click to launch
import subprocess, os, time, json
from IPython.display import display, HTML

STATE = os.environ.get('OPENCLAW_STATE_DIR', '/content/openclaw_state')

# Preflight checks
oc_bin = subprocess.run(['which', 'openclaw'], capture_output=True, text=True).stdout.strip()
if not oc_bin:
    display(HTML('<div style="background:#f8d7da; color:#721c24; padding:12px; border-radius:8px;">❌ OpenClaw not found. Run <b>Step 2</b> first.</div>'))
elif not os.environ.get('OPENCLAW_GATEWAY_TOKEN'):
    display(HTML('<div style="background:#f8d7da; color:#721c24; padding:12px; border-radius:8px;">❌ No keys configured. Run <b>Step 2</b> first.</div>'))
else:
    # Write openclaw.json — auth + enable HTTP chat API + Control UI dashboard
    # reload.mode=hot: don't force-restart when the file watcher detects config changes
    config = {
        'gateway': {
            'auth': {
                'token': os.environ['OPENCLAW_GATEWAY_TOKEN']
            },
            'http': {
                'endpoints': {
                    'chatCompletions': {
                        'enabled': True
                    }
                }
            },
            'controlUi': {
                'enabled': True
            },
            'reload': {
                'mode': 'hot'
            }
        }
    }

    # Apply model selection (from the "Choose a Model" cell, or skip for auto-detect)
    # agents.defaults.model must be an object { primary: "provider/model-id" }, not a plain string
    selected_model = os.environ.get('OPENCLAW_SELECTED_MODEL')
    if selected_model:
        config['agents'] = {
            'defaults': {
                'model': {
                    'primary': selected_model
                }
            }
        }

    config_path = os.path.join(STATE, 'openclaw.json')
    with open(config_path, 'w') as f:
        json.dump(config, f, indent=2)

    # Pre-create workspace dir (where MEMORY.md lives)
    os.makedirs(os.path.join(STATE, 'workspace'), exist_ok=True)

    # Environment
    env = os.environ.copy()
    env['OPENCLAW_STATE_DIR'] = STATE
    env['HOME'] = '/content'
    env['NODE_ENV'] = 'production'
    env['OPENCLAW_SKIP_CHANNELS'] = '1'
    env['OPENCLAW_SKIP_GMAIL_WATCHER'] = '1'
    env['OPENCLAW_NO_RESPAWN'] = '1'  # in-process restart instead of process exit (no systemd on Colab)

    # Symlink ~/.openclaw -> state dir
    link = '/content/.openclaw'
    if os.path.islink(link):
        os.unlink(link)
    if not os.path.exists(link):
        os.symlink(STATE, link)

    # Stop previous instance
    subprocess.run(['pkill', '-f', 'openclaw.*gateway'], capture_output=True)
    time.sleep(2)

    # Launch
    log = open('/content/openclaw_gateway.log', 'w')
    proc = subprocess.Popen(
        [oc_bin, 'gateway', '--allow-unconfigured', '--bind', 'lan', '--port', '18789'],
        env=env, stdout=log, stderr=subprocess.STDOUT, preexec_fn=os.setsid,
    )

    display(HTML('<b style="color:#333">⏳ Starting OpenClaw...</b>'))
    time.sleep(6)

    if proc.poll() is None:
        token_preview = os.environ.get('OPENCLAW_GATEWAY_TOKEN', '')[:16]

        # Read the log to find what model OpenClaw picked
        model_line = ''
        try:
            with open('/content/openclaw_gateway.log') as f:
                for line in f:
                    if 'agent model:' in line:
                        model_line = line.split('agent model:')[1].strip()
                        break
        except Exception:
            pass
        model_display = model_line or selected_model or 'auto-detected'

        display(HTML(f'''
        <div style="background:#d4edda; color:#155724; border:1px solid #28a745; border-radius:8px; padding:16px; margin:8px 0;">
          <h3 style="margin:0 0 10px 0; color:#155724;">✅ OpenClaw is running!</h3>
          <table style="border:none; border-collapse:collapse; font-size:14px; color:#1a1a1a;">
            <tr><td style="padding:3px 12px 3px 0"><b>Model</b></td><td><code style="color:#2e7d32">{model_display}</code></td></tr>
            <tr><td style="padding:3px 12px 3px 0"><b>Dashboard</b></td><td><a href="http://localhost:18789/ui/" target="_blank" style="color:#2e7d32">http://localhost:18789/ui/</a></td></tr>
            <tr><td style="padding:3px 12px 3px 0"><b>Logs</b></td><td><code style="color:#2e7d32">/content/openclaw_gateway.log</code></td></tr>
            <tr><td style="padding:3px 12px 3px 0"><b>Token</b></td><td><code style="color:#2e7d32">{token_preview}...</code></td></tr>
          </table>
          <p style="margin:10px 0 0 0; font-size:13px; color:#333;">
            Scroll down to <b>"What now?"</b> to see your options.
          </p>
        </div>
        '''))
    else:
        with open('/content/openclaw_gateway.log') as f:
            err = f.read()[-3000:]
        display(HTML(f'''
        <div style="background:#f8d7da; color:#721c24; border:1px solid #dc3545; border-radius:8px; padding:16px;">
          <b>❌ Gateway failed to start.</b>
          <pre style="margin-top:8px; font-size:12px; max-height:300px; overflow:auto; color:#721c24;">{err}</pre>
        </div>
        '''))

---
## What now?

Your OpenClaw gateway is running inside this Colab VM, but **Colab has no public IP** — nothing from the outside internet can reach it directly.

| Option | What it does | Needs a tunnel? |
|---|---|---|
| **💬 Chat UI** | Gradio chat right here in the notebook | ❌ No — runs locally inside the same VM |
| **🖥️ Dashboard** | OpenClaw's built-in web dashboard at `localhost:18789/ui/` | ❌ No — works in this Colab tab. With a tunnel, accessible from any device |
| **🌐 ngrok tunnel** | Creates a public HTTPS URL that routes traffic into the Colab VM | — (this *is* the tunnel) |

> 💡 **Just want to chat?** Run the **Chat UI** cell right below — it works immediately, no tunnel needed.
>
> 🌐 **Want to use OpenClaw from your phone, connect messaging channels, or call the API from other apps?** You'll need to set up the **ngrok tunnel** further down.

---

### ⚠️ If something goes wrong

Colab is a free, ephemeral environment — things can and do break. Here's how to recover:

| Symptom | What to do |
|---|---|
| **Chat returns "Gateway not running"** | Run the 🟢 **Check status** utility cell. If it says "Not running", re-run **Step 3** to relaunch. |
| **Gateway crashed or timed out** | Run ⏹️ **Stop gateway** first (to clean up), then re-run **Step 3**. Your config and secrets are preserved. |
| **"Drain timeout" or "shutdown timed out" in logs** | This happens when the gateway restarts internally (e.g., after a config change via the dashboard). It should recover on its own — wait 10–15 seconds and try again. If it doesn't, stop + relaunch (Step 3). |
| **Slow or hanging responses** | Some models (especially large ones like Opus or GPT-5.2 Pro) can take 30–60s on long prompts. Try a faster model (Gemini Flash, Haiku, GPT-5 Nano). |
| **Colab disconnected / runtime recycled** | Normal — free Colab runtimes shut down after ~12h of inactivity. Re-run Steps 2 → 3. If you enabled Drive persistence, your conversations are saved. |
| **"Pairing required" errors in logs** | Safe to ignore — this is the gateway's WebSocket security handshake. It resolves automatically. |

> 💡 **Quick recovery shortcut:** Run these two cells in order: ⏹️ **Stop gateway** → ▶️ **Step 3 (launch)**. This fixes most issues.

---
### 💬 Chat UI (Gradio)

A simple chat interface that runs inside the notebook. Uses the OpenAI-compatible API built into the gateway.

In [ ]:
#@title ▶️ Launch chat
!pip install gradio -q

import gradio as gr, requests, os, json

URL = 'http://localhost:18789'
TOK = os.environ.get('OPENCLAW_GATEWAY_TOKEN', '')

def chat(message, history):
    """Send a message via OpenClaw's OpenAI-compatible endpoint."""
    # Build messages from history + new message
    messages = []
    for entry in (history or []):
        role = entry.get('role', 'user') if isinstance(entry, dict) else 'user'
        content = entry.get('content', str(entry)) if isinstance(entry, dict) else str(entry)
        messages.append({'role': role, 'content': content})
    messages.append({'role': 'user', 'content': message})

    try:
        r = requests.post(
            f'{URL}/v1/chat/completions',
            headers={
                'Content-Type': 'application/json',
                'Authorization': f'Bearer {TOK}',
            },
            json={
                'model': 'openclaw',
                'messages': messages,
                'stream': False,
            },
            timeout=120,
        )
        if r.status_code == 200:
            data = r.json()
            choices = data.get('choices', [])
            if choices:
                return choices[0].get('message', {}).get('content', str(data))
            return str(data)
        return f'Error {r.status_code}: {r.text[:500]}'
    except requests.exceptions.ConnectionError:
        return '❌ Gateway not running. Run Step 3 first.'
    except requests.exceptions.Timeout:
        return '⏳ Request timed out. The model may be processing a complex query.'
    except Exception as e:
        return f'Error: {e}'

gr.ChatInterface(
    fn=chat,
    type='messages',
    title='🦞 CoLobster Chat',
    examples=['Hello! What can you do?', 'Summarize the benefits of open-source AI gateways.'],
).launch(share=False, debug=False)

---
### 🌐 Public Tunnel (ngrok)

Google Colab VMs have **no public IP address** — external traffic simply cannot reach the gateway. A tunnel solves this by creating a public URL that routes into this VM.

#### Without a tunnel (default)

| Feature | Works? |
|---|---|
| **💬 Gradio Chat UI** (in this notebook) | ✅ Yes — runs in the same VM |
| **🖥️ Dashboard** at `localhost:18789/ui/` (this Colab tab only) | ✅ Yes — same VM |
| Access from your phone or another device | ❌ No |
| OpenAI-compatible API from external apps | ❌ No |
| Webhook channels (Slack HTTP, Telegram webhook) | ❌ No |

#### With a tunnel

Everything above **plus**:

| Feature | Works? |
|---|---|
| **🖥️ Dashboard** from any device (phone, laptop) | ✅ Yes |
| **OpenAI-compatible API** (`/v1/chat/completions`) from external apps | ✅ Yes |
| **Slack** (HTTP webhook mode) | ✅ Yes |
| **Telegram** (webhook mode) | ✅ Yes |
| **WebSocket** connections (CLI/SDK clients, real-time dashboard updates) | ✅ Yes |

#### Channels that work WITHOUT a tunnel

Some messaging channels use **outbound** connections (the gateway connects to them, not the other way around):

- **Discord** — socket mode (outbound WebSocket)
- **Telegram** — polling mode (outbound HTTP, no webhook needed)
- **Slack** — socket mode (outbound WebSocket, no HTTP webhook needed)

These work behind Colab's NAT without any tunnel. Configure them via `openclaw.json` or the dashboard.

#### Setup

1. Get a free auth token at [ngrok.com](https://ngrok.com)
2. Add `NGROK_AUTH_TOKEN` to the 🔑 Secrets panel
3. Run the cell below

> ⚠️ **ngrok free tier** gives a random URL that changes every time you restart the tunnel. For a stable URL, consider [Cloudflare Tunnel](https://developers.cloudflare.com/cloudflare-one/connections/connect-networks/) (`cloudflared`) as a free alternative.

In [ ]:
#@title ▶️ Open ngrok tunnel
import os
from IPython.display import display, HTML

tok = None
try:
    from google.colab import userdata
    tok = userdata.get('NGROK_AUTH_TOKEN')
except Exception:
    tok = os.environ.get('NGROK_AUTH_TOKEN')

if not tok:
    display(HTML('''
    <div style="background:#fff3cd; color:#856404; border:1px solid #ffc107; border-radius:8px; padding:14px;">
      <b>⚠️</b> Add <code style="color:#856404">NGROK_AUTH_TOKEN</code> to the 🔑 Secrets panel, then re-run.
    </div>
    '''))
else:
    import subprocess
    subprocess.run(['pip', 'install', 'pyngrok', '-q'], check=True)
    from pyngrok import ngrok, conf
    conf.get_default().auth_token = tok.strip()
    tunnel = ngrok.connect(18789, 'http')
    public_url = tunnel.public_url

    # Write the public URL into the agent's MEMORY.md so the bot knows its own address.
    # OpenClaw loads MEMORY.md into the agent's system prompt and indexes it for memory_search.
    workspace_dir = os.path.join(os.environ.get('OPENCLAW_STATE_DIR', '/content/openclaw_state'), 'workspace')
    os.makedirs(workspace_dir, exist_ok=True)
    memory_path = os.path.join(workspace_dir, 'MEMORY.md')

    # Read existing memory (if any) and update the public URL section
    existing = ''
    try:
        with open(memory_path) as f:
            existing = f.read()
    except FileNotFoundError:
        pass

    # Remove any previous public URL section
    import re
    existing = re.sub(r'## Public Gateway URL\n.*?(?=\n## |\Z)', '', existing, flags=re.DOTALL).strip()

    url_section = f"""## Public Gateway URL

The gateway's public URL is: {public_url}
This ngrok tunnel routes external traffic into the Colab VM.
Use this URL for webhook registrations, sharing with external apps, or accessing the dashboard from other devices.
The dashboard is at: {public_url}/ui/
The OpenAI-compatible chat API is at: {public_url}/v1/chat/completions
"""

    with open(memory_path, 'w') as f:
        content = f"{existing}\n\n{url_section}".strip() + '\n' if existing else url_section
        f.write(content)

    display(HTML(f'''
    <div style="background:#d4edda; color:#155724; border:1px solid #28a745; border-radius:8px; padding:14px;">
      ✅ <b>Public URL:</b> <a href="{public_url}" target="_blank" style="color:#155724">{public_url}</a><br>
      <span style="font-size:13px; color:#1a1a1a;">
        Dashboard: <a href="{public_url}/ui/" target="_blank" style="color:#2e7d32">{public_url}/ui/</a><br>
        The agent now knows this URL (written to MEMORY.md).
      </span>
    </div>
    '''))

---
### 🛠️ Utilities

In [ ]:
#@title 📜 View logs
!tail -50 /content/openclaw_gateway.log 2>/dev/null || echo 'No log file yet.'

In [ ]:
#@title 🟢 Check status
!pgrep -fa 'openclaw.*gateway' || echo '❌ Not running. Run Step 3.'

In [ ]:
#@title ⏹️ Stop gateway
!pkill -f 'openclaw.*gateway' && echo '✅ Stopped.' || echo 'Not running.'